<a href="https://colab.research.google.com/github/luvsgna/BTECH_PROJECT/blob/main/INCEPTION_COMPLETE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define paths to your dataset stored in Google Drive
train_dir = '/content/drive/MyDrive/deepfake_database/train'
val_dir = '/content/drive/MyDrive/deepfake_database/validation'
test_dir = '/content/drive/MyDrive/deepfake_database/test'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Define data augmentation for training data
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define data generator for validation and test data (only normalization)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Load the datasets from Google Drive paths
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 12033 images belonging to 2 classes.
Found 7114 images belonging to 2 classes.
Found 330 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the InceptionV3 model without the top layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Final model for training
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers initially
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Show model summary
model.summary()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 150, 150, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 74, 74, 32)     │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 74, 74, 32)     │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 74, 74, 32)     │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 72, 72, 32)     │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 72, 72, 32)     │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 72, 72, 32)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 72, 72, 64)     │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 72, 72, 64)     │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 72, 72, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 35, 35, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 35, 35, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 35, 35, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 35, 35, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 33, 33, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 33, 33, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 23,901,985 (91.18 MB)

 Trainable params: 2,099,201 (8.01 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [ ]:
# Train the model on training data and validate on validation data
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 1060s 3s/step - accuracy: 0.9121 - loss: 0.2172 - val_accuracy: 0.8652 - val_loss: 0.3128
Epoch 2/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 1048s 3s/step - accuracy: 0.9064 - loss: 0.2173 - val_accuracy: 0.8388 - val_loss: 0.3609
Epoch 3/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 1080s 3s/step - accuracy: 0.9068 - loss: 0.2243 - val_accuracy: 0.8690 - val_loss: 0.3024
Epoch 4/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 1046s 3s/step - accuracy: 0.9189 - loss: 0.2003 - val_accuracy: 0.8759 - val_loss: 0.3049
Epoch 5/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 999s 3s/step - accuracy: 0.9173 - loss: 0.1980 - val_accuracy: 0.8821 - val_loss: 0.2972
Epoch 6/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 1005s 3s/step - accuracy: 0.9225 - loss: 0.1862 - val_accuracy: 0.8728 - val_loss: 0.2974
Epoch 7/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 1044s 3s/step - accuracy: 0.9242 - loss: 0.1850 - val_accuracy: 0.8742 - val_loss: 0.3068
Epoch 8/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 1023s 3s/step - accuracy: 0.9261 - loss: 0.1805 - v

In [ ]:
# Access the final training accuracy from the history object
train_accuracy = history.history['accuracy'][-1]  # Get the last epoch's accuracy
print(f"Training Accuracy: {train_accuracy:.2f}")

Training Accuracy: 0.93


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Get predictions on the test set
predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype("int32")  # Convert probabilities to binary 0/1

# Get the true labels
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Print classification report
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Calculate and print test accuracy
test_accuracy = accuracy_score(true_classes, predicted_classes)
print(f"Calculated Test Accuracy: {test_accuracy:.2f}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step
Classification Report:
              precision    recall  f1-score   support

          df       0.89      0.85      0.87       165
        real       0.86      0.90      0.88       165

    accuracy                           0.87       330
   macro avg       0.87      0.87      0.87       330
weighted avg       0.87      0.87      0.87       330

Calculated Test Accuracy: 0.87


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Predict on the test set
predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype("int32")  # Convert probabilities to binary 0/1

# True labels
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Test Accuracy
test_accuracy = accuracy_score(true_classes, predicted_classes)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Precision, Recall, and F1-Score
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Confusion Matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print(f"\nConfusion Matrix:\n{conf_matrix}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step
Test Accuracy: 0.87

Classification Report:
              precision    recall  f1-score   support

          df       0.89      0.85      0.87       165
        real       0.86      0.90      0.88       165

    accuracy                           0.87       330
   macro avg       0.87      0.87      0.87       330
weighted avg       0.87      0.87      0.87       330

Precision: 0.86
Recall: 0.90
F1-Score: 0.88

Confusion Matrix:
[[140  25]
 [ 17 148]]
